# Table of Contents #
### 0. Introduction & Initialization ###
### 1. Data Loading ###
### 2. Data Cleaning ###
### 3. Data Analysis ###
### 4. Dataset Creation ###
### 5. Visualization ###

# 0. Introduction & Initialization #

Hello! This documentation was written by Colin Vu as of 5/16/2025. Any questions can be sent to cvu33@gatech.edu

The intent of the CountyBuddy program is to create a simplified way for users to download "context data" for tracts and counties. It parses US Census and American Community Survey data for various demographics to find outlier data.

Users may select between tract and county data. The output file produced by CountyBuddy contains the following information. More detailed descriptions of each data category are available in its respective section.
- A record for each tract/county
- The identification codes for the tract/county
- The total population of the tract/county
- The population of each demographic within the tract/county
- The ratio of that demographic to the total population
For any demographic group where the outlier threshold is not met, the population and ratio columns are zeroed out.

The Census and ACS data used can be downloaded online via https://data.census.gov/. Datasets downloaded through here can be directly added to CountyBuddy's addl_data folder and will be automatically parsed by CountyBuddy if you follow the directions located in the addl_data folder's README.txt file.

Each output file contains the addl_data demographic data as well as default data listed below. The existing addl_data demographic data also serves as default data, but is removeable.

To run CountyBuddy, simply run this Jupyter Notebook program in its entirely. Note that section 5 (Visualization) is not currently functional - the output files are created in section 4 (Dataset Creation). The full process should take about 5 minutes.

Libraries and basic variables are initialized here.

The method which meaningful context data is determined is based on the inclusion factor. The inclusion factor is the number of standard deviations above the mean for which data will be used. Only data above that level is included. This number can be changed as needed.

In [198]:
inclusion_factor = 2

In [199]:
import pandas as pd
import requests 
import geopandas as gpd
from io import StringIO
import sys
import folium
import numpy as np
from shapely.geometry import Point
import ipywidgets as widgets
import geojson
import json

names = []
ratios = []
newNames = []
newRatios = []

# 1. Data Loading #

Here, shapefile and CSV data are loaded. If you would like to change the files which are read, then you can trace their filepaths from here.

## a. Shapefiles ##

In [200]:
# Initialize USA Counties Shapefile

#simplification method: Visvalingam / weighted area, 25% simplified 

# Paths
countiesPath = '../cb_2020_us_county_500k/cb_2020_us_county_500k.shp'

# Load shapefiles
counties = gpd.read_file(countiesPath)
print(counties.head())
counties = counties[['STATE_NAME','AFFGEOID','GEOID','NAMELSAD','geometry']]

  STATEFP COUNTYFP  COUNTYNS        AFFGEOID  GEOID      NAME  \
0      13      031  00350302  0500000US13031  13031   Bulloch   
1      13      121  01694833  0500000US13121  13121    Fulton   
2      13      179  00357095  0500000US13179  13179   Liberty   
3      13      189  00348794  0500000US13189  13189  McDuffie   
4      13      213  01688005  0500000US13213  13213    Murray   

          NAMELSAD STUSPS STATE_NAME LSAD       ALAND     AWATER  \
0   Bulloch County     GA    Georgia   06  1750735216   32827779   
1    Fulton County     GA    Georgia   06  1364133162   19666288   
2   Liberty County     GA    Georgia   06  1337654102  222884791   
3  McDuffie County     GA    Georgia   06   666590014   23114032   
4    Murray County     GA    Georgia   06   892235145    5719644   

                                            geometry  
0  POLYGON ((-82.02684 32.55516, -82.02527 32.558...  
1  POLYGON ((-84.84931 33.51318, -84.84429 33.514...  
2  POLYGON ((-81.8244 32.01488, -81

In [201]:
print(counties.head())

  STATE_NAME        AFFGEOID  GEOID         NAMELSAD  \
0    Georgia  0500000US13031  13031   Bulloch County   
1    Georgia  0500000US13121  13121    Fulton County   
2    Georgia  0500000US13179  13179   Liberty County   
3    Georgia  0500000US13189  13189  McDuffie County   
4    Georgia  0500000US13213  13213    Murray County   

                                            geometry  
0  POLYGON ((-82.02684 32.55516, -82.02527 32.558...  
1  POLYGON ((-84.84931 33.51318, -84.84429 33.514...  
2  POLYGON ((-81.8244 32.01488, -81.81338 32.0162...  
3  POLYGON ((-82.64852 33.60838, -82.64409 33.606...  
4  POLYGON ((-84.94434 34.68004, -84.9431 34.6804...  


In [202]:
#Initialize USA Census Tract Shapefile https://www.census.gov/geographies/mapping-files/time-series/geo/cartographic-boundary.html 

#Path 1:500,000
#complexTractPath = '../cb_2022_us_tract_500k/cb_2022_us_tract_500k.shp'

#Path 1:5,000,000
tractPath = '../cb_2020_us_tract_500k/cb_2020_us_tract_500k.shp'

#Load Shapefile
tracts = gpd.read_file(tractPath)
print(tracts)
tracts = tracts[['STATE_NAME','AFFGEOID', 'NAMELSAD', 'NAMELSADCO','geometry']]

      STATEFP COUNTYFP TRACTCE              AFFGEOID        GEOID     NAME  \
0          06      077  003406  1400000US06077003406  06077003406    34.06   
1          06      077  004402  1400000US06077004402  06077004402    44.02   
2          06      077  004600  1400000US06077004600  06077004600       46   
3          06      077  001400  1400000US06077001400  06077001400       14   
4          06      077  003106  1400000US06077003106  06077003106    31.06   
...       ...      ...     ...                   ...          ...      ...   
85182      36      027  050104  1400000US36027050104  36027050104   501.04   
85183      24      047  950100  1400000US24047950100  24047950100     9501   
85184      17      043  841310  1400000US17043841310  17043841310  8413.10   
85185      17      043  846410  1400000US17043846410  17043846410  8464.10   
85186      17      043  845508  1400000US17043845508  17043845508  8455.08   

                   NAMELSAD STUSPS          NAMELSADCO  STATE_N

In [203]:
print(tracts.head())

   STATE_NAME              AFFGEOID            NAMELSAD          NAMELSADCO  \
0  California  1400000US06077003406  Census Tract 34.06  San Joaquin County   
1  California  1400000US06077004402  Census Tract 44.02  San Joaquin County   
2  California  1400000US06077004600     Census Tract 46  San Joaquin County   
3  California  1400000US06077001400     Census Tract 14  San Joaquin County   
4  California  1400000US06077003106  Census Tract 31.06  San Joaquin County   

                                            geometry  
0  POLYGON ((-121.309 38.02824, -121.30461 38.028...  
1  POLYGON ((-121.27338 38.10811, -121.27286 38.1...  
2  POLYGON ((-121.411 38.23193, -121.40976 38.232...  
3  POLYGON ((-121.30988 37.98438, -121.30745 37.9...  
4  POLYGON ((-121.37304 37.99888, -121.37139 38.0...  


## b. CSV files ##

In [204]:
#Load county population csv 
county_pop = pd.read_csv('../county_population.csv')
#print(county_pop.head())

In [205]:
#Load tract population csv
tract_pop = pd.read_csv('../Decentenial2020PopulationbyTract/DECENNIALDP2020.DP1-Data.csv', low_memory=False)

In [206]:
#University and Colleges CSV
universities = pd.read_csv('../CollegesUniversities2.csv', low_memory=False)
#print(universities.head())


In [207]:
#load TRACT group quarters csv
p5_tract_data = pd.read_csv('../DECENNIALPL2020.P5_Tracts/DECENNIALPL2020.P5-Data.csv', low_memory=False)
#print(p5_tract_data.head())

In [208]:
#load COUNTIES group quarters csv (TO-DO)
p5_data = pd.read_csv('../DECENNIALPL2020.P5_2024-02-19T170254/DECENNIALPL2020.P5-Data.csv', low_memory=False)


In [209]:
#Load ACS economics csv
ACS_data = pd.read_csv('../ACS_Economic_data/ACS_Economic_data.csv', low_memory=False)

In [210]:
#New column data, gathered from the US census bureau
new_cols_tract = pd.read_csv('../addl_data/addl_data_tract.csv')
col_names_tract = new_cols_tract.columns.tolist()
col_names_tract = col_names_tract[2:]

new_cols_county = pd.read_csv('../addl_data/addl_data_county.csv', low_memory=False)

native_county = pd.read_csv('../addl_data/native_american_county.csv', low_memory=False)
new_cols_county = pd.merge(new_cols_county, native_county, left_on="NAME", right_on="COUNTY", how="inner")
new_cols_county = new_cols_county.drop(columns=["COUNTY"])

col_names_county = new_cols_county.columns.tolist()
col_names_county = col_names_county[3:]

# 2. Data Cleaning #

This is where data is cleaned so that it can be exported and visualized. Note that the implementation of the cleaning process is a little messy--in future iterations of this program, this may be fixed for reading clarity and maintenance purposes.

In [211]:
county_pop_2 = pd.read_csv('../county_pop.csv')
county_pop_2['County_Name'] = county_pop_2['NAME']
county_pop_2['2020_base'] = county_pop_2['P0010001']
county_pop_2 = county_pop_2.rename(columns={'GEOID': 'GEO_ID'})
county_pop_2['GEO_ID'] = county_pop_2['GEO_ID'].astype(str)
county_pop_2['GEO_ID'] = county_pop_2['GEO_ID'].str.zfill(5)


In [212]:
finalTract = pd.read_csv('../tract_data_final.csv')
finalTract['CONCATENATE_featurenam'] = finalTract['CONCATENATE_featurenam'].astype(str).apply(lambda x: '; '.join(sorted(set(x.split('; ')), key=x.split('; ').index)))
finalTract['CONCATENATE_featurenam'] = finalTract['CONCATENATE_featurenam'].replace({pd.NA: "", "nan": "", "NaN": "", None: ""}).fillna("")
finalTract['CONCATENATE_IND_NAME'] = finalTract['CONCATENATE_IND_NAME'].astype(str).apply(lambda x: '; '.join(sorted(set(x.split('; ')), key=x.split('; ').index)))
finalTract['CONCATENATE_IND_NAME'] = finalTract['CONCATENATE_IND_NAME'].replace({pd.NA: "", "nan": "", "NaN": "", None: ""}).fillna("")
finalTract.to_csv('tract_data_final_cleaned.csv', index=False)

In [213]:
#Clean County Population Csv
new_column_names = ['County_Name'] + county_pop.iloc[0, 1:].tolist()

# Assign the new column names to the DataFrame
county_pop.columns = new_column_names

# Drop the first row (now redundant since it was used for column names)
county_pop = county_pop.drop(county_pop.index[0])

# Reset the index if necessary
county_pop = county_pop.reset_index(drop=True)


county_pop['County_Name'] = county_pop['County_Name'].str.lstrip('.')

county_pop = county_pop.rename(columns={'County_Name': 'NAME'})

county_pop['2022'] = county_pop['2022'].str.replace(',', '')

county_pop['2020_base'] = county_pop['2020_base'].str.replace(',', '')

county_pop['2022' ]= pd.to_numeric(county_pop['2022'], errors='coerce')

county_pop['2020_base' ]= pd.to_numeric(county_pop['2020_base'], errors='coerce')

print(county_pop.head())

                      NAME  2020_base     2020     2021      2022
0  Autauga County, Alabama    58802.0   58,902   59,210   59759.0
1  Baldwin County, Alabama   231761.0  233,219  239,361  246435.0
2  Barbour County, Alabama    25224.0   24,960   24,539   24706.0
3     Bibb County, Alabama    22300.0   22,183   22,370   22005.0
4   Blount County, Alabama    59130.0   59,102   59,085   59512.0


In [214]:
#Clean Tract population data 

# Find the index of the column up to which you want to keep the data
column_index = tract_pop.columns.get_loc('DP1_0001C') + 1  # add 1 because slicing is exclusive on the stop

# Keep only the columns up to and including 'DP1_0001C'
tract_pop = tract_pop.iloc[:, :column_index]

# Drop the first row (now redundant since it was used for column names)
tract_pop = tract_pop.drop(tract_pop.index[0])

# Reset the index if necessary
tract_pop = tract_pop.reset_index(drop=True)

tract_pop['DP1_0001C' ]= pd.to_numeric(tract_pop['DP1_0001C'], errors='coerce')

tract_pop = tract_pop.rename(columns={'DP1_0001C': 'total_pop'})

print(tract_pop.head())

                 GEO_ID                                          NAME  \
0  1400000US01001020100     Census Tract 201; Autauga County; Alabama   
1  1400000US01001020200     Census Tract 202; Autauga County; Alabama   
2  1400000US01001020300     Census Tract 203; Autauga County; Alabama   
3  1400000US01001020400     Census Tract 204; Autauga County; Alabama   
4  1400000US01001020501  Census Tract 205.01; Autauga County; Alabama   

   total_pop  
0       1775  
1       2055  
2       3216  
3       4246  
4       4322  


In [215]:
#Clean University CSV
universities = universities[universities['NAICS_DESC'] == 'COLLEGES, UNIVERSITIES, AND PROFESSIONAL SCHOOLS']
universities = universities[universities['DORM_CAP'] > 500]

# Assuming 'universities' DataFrame has 'latitude' and 'longitude' columns
geometry = [Point(xy) for xy in zip(universities['LONGITUDE'], universities['LATITUDE'])]
universities_gdf = gpd.GeoDataFrame(universities, geometry=geometry)
universities_gdf = universities_gdf[['NAME','ADDRESS','CITY','STATE','COUNTYFIPS','FT_ENROLL','PT_ENROLL','DORM_CAP','geometry']]
universities_gdf.rename(columns={'NAME':'UNIVERSITY_NAME'}, inplace=True)

# Ensure CRS is set; assuming WGS84 for lat/lon coordinates
universities_gdf.crs = "EPSG:4269"


In [216]:
print(universities_gdf.columns)

Index(['UNIVERSITY_NAME', 'ADDRESS', 'CITY', 'STATE', 'COUNTYFIPS',
       'FT_ENROLL', 'PT_ENROLL', 'DORM_CAP', 'geometry'],
      dtype='object')


In [217]:
#Clean Group Quarters Tracts CSV
# Delete the first row
p5_tract_data = p5_tract_data.drop(index=0)
# Reset the index
p5_tract_data = p5_tract_data.reset_index(drop=True)


In [218]:
#Clean new_cols
new_cols_tract = new_cols_tract.drop(index=0)
# Reset the index
new_cols_tract = new_cols_tract.reset_index(drop=True)

In [219]:
#merge tract population table with group quarters tracts data 
p5_tract_data_merged = p5_tract_data.merge(tract_pop[['GEO_ID', 'total_pop']], on='GEO_ID', how='left')
p5_tract_data_merged['total_pop'] = p5_tract_data_merged['total_pop'].fillna(0)

for col in col_names_tract:
  p5_tract_data_merged = p5_tract_data_merged.merge(new_cols_tract[['GEO_ID', col]], on='GEO_ID', how='left')
  p5_tract_data_merged[col] = p5_tract_data_merged[col].fillna(0)

In [220]:
p5_tract_data_merged.rename(columns={'GEO_ID':'AFFGEOID'},inplace=True)

In [221]:
# Ensure both IDs are of the same data type, e.g., string
p5_tract_data_merged['AFFGEOID'] = p5_tract_data_merged['AFFGEOID'].astype(str)
tracts['AFFGEOID'] = tracts['AFFGEOID'].astype(str)

# List of column names you want to convert to numeric
columns_to_convert = [
    'P5_001N', 'P5_002N', 'P5_003N', 'P5_004N', 'P5_005N',
    'P5_006N', 'P5_007N', 'P5_008N', 'P5_009N', 'P5_010N', 'P5_011N'
]
columns_to_convert.extend(col_names_tract)

# Convert each specified column to numeric, coercing errors
for column in columns_to_convert:
    p5_tract_data_merged[column] = pd.to_numeric(p5_tract_data_merged[column], errors='coerce')


#print(p5_tract_data_merged.head())

In [222]:
#Clean Group Quarters County Data

# Delete the first row
p5_data = p5_data.drop(index=0)

# Reset the index
p5_data = p5_data.reset_index(drop=True)

p5_data['GEO_ID'] = p5_data['GEO_ID'].str[-5:]

# Ensure both IDs are of the same data type, e.g., string
p5_data['GEO_ID'] = p5_data['GEO_ID'].astype(str)
counties['GEOID'] = counties['GEOID'].astype(str)
counties.rename(columns={'GEOID': 'GEO_ID'}, inplace=True)

#Convert desired columns to numeric
p5_data['P5_003N'] = pd.to_numeric(p5_data['P5_003N'], errors='coerce')
p5_data['P5_007N'] = pd.to_numeric(p5_data['P5_007N'], errors='coerce')
p5_data['P5_008N'] = pd.to_numeric(p5_data['P5_008N'], errors='coerce')
p5_data['P5_009N'] = pd.to_numeric(p5_data['P5_009N'], errors='coerce')

print(p5_data.head())

  GEO_ID                     NAME P5_001N P5_002N  P5_003N P5_004N P5_005N  \
0  01001  Autauga County, Alabama     487     408      142       6     260   
1  01003  Baldwin County, Alabama    3552    2761     1229      74    1212   
2  01005  Barbour County, Alabama    2717    2672     2517       0     155   
3  01007     Bibb County, Alabama    2140    2101     1856      10     235   
4  01009   Blount County, Alabama     613     411       91       0     320   

  P5_006N  P5_007N  P5_008N  P5_009N P5_010N  
0       0       79        0        0      79  
1     246      791      309        0     482  
2       0       45        0        0      45  
3       0       39        0        0      39  
4       0      202        0        0     202  


***ACS Columns:*** 
- **DP03_0033E, DP03_0033M:** Estimate & Margin error of Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining
- **DP03_0035E, DP03_0035M** Estimate & Margin of error Civilian employed population 16 years and over!!Manufacturing
- **DP03_0033PE, DP03_0033PM:** Percent & Margin error percent of Civilian employed population 16 years and over!!Agriculture, forestry, fishing and hunting, and mining

In [223]:
#Clean ACS data 

#keep columns GEO_ID, NAME, DP03_0033E, DP03_0033M, DP03_0035E, DP03_0035M, DP03_0033PE, DP03_0033PM
ACS_data = ACS_data[['GEO_ID', 'NAME', 'DP03_0033E', 'DP03_0033M', 'DP03_0035E', 'DP03_0035M', 'DP03_0033PE', 'DP03_0033PM']]
ACS_data.rename(columns={'GEO_ID':'AFFGEOID'},inplace=True)

# Delete the first row
ACS_data = ACS_data.drop(index=0)
#ResetIndex
ACS_data.reset_index(drop=True, inplace=True)
#change values to floats
ACS_data['DP03_0033PE'] = pd.to_numeric(ACS_data['DP03_0033PE'], errors='coerce')
#replace NA values with 0
ACS_data['DP03_0033PE'] = ACS_data['DP03_0033PE'].fillna(0)

In [224]:
print(ACS_data.head())

               AFFGEOID                                          NAME  \
0  1400000US01001020100     Census Tract 201; Autauga County; Alabama   
1  1400000US01001020200     Census Tract 202; Autauga County; Alabama   
2  1400000US01001020300     Census Tract 203; Autauga County; Alabama   
3  1400000US01001020400     Census Tract 204; Autauga County; Alabama   
4  1400000US01001020501  Census Tract 205.01; Autauga County; Alabama   

  DP03_0033E DP03_0033M DP03_0035E DP03_0035M  DP03_0033PE DP03_0033PM  
0          2          3         75         60          0.3         0.4  
1          0         13        114         52          0.0         4.2  
2          0         13        183         98          0.0         2.1  
3          3          5        347        219          0.2         0.3  
4          0         13        126         94          0.0         1.6  


# 3. Data Analysis #


Here, part of the process of parsing data to select the right data to export is done. This section is also messy, and may be changed for maintenance and reading purposes.

In [225]:
#Getting military population ratios for TRACT data
#total_military_ratio = military group quarters population / total tract population
p5_tract_data_merged['total_military_ratio'] = np.divide(p5_tract_data_merged['P5_009N'], p5_tract_data_merged['total_pop'])
p5_tract_data_merged['total_military_ratio'] = p5_tract_data_merged['total_military_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)


#military_ratio = ratio of military population to total non-intstitutionalized population
p5_tract_data_merged['relative_military_ratio'] = np.divide(p5_tract_data_merged['P5_009N'], p5_tract_data_merged['P5_007N'])
# Replace inf/-inf and NA values with 0 in the new column
p5_tract_data_merged['relative_military_ratio'] = p5_tract_data_merged['relative_military_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

In [226]:
#Getting Incarcerated Population ratios for TRACT data

#total_incarcerated_ratio = incarcerated population / total tract pop 
p5_tract_data_merged['total_incarcerated_ratio'] = np.divide(p5_tract_data_merged['P5_003N'], p5_tract_data_merged['total_pop'])
p5_tract_data_merged['total_incarcerated_ratio'] = p5_tract_data_merged['total_incarcerated_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)


#print(p5_tract_data_merged.head())

In [227]:
#Getting reservation population ratios for TRACT data
#total_military_ratio = military group quarters population / total tract population
p5_tract_data_merged['reservation_ratio'] = np.divide(p5_tract_data_merged['P5_011N'], p5_tract_data_merged['total_pop'])
p5_tract_data_merged['reservation_ratio'] = p5_tract_data_merged['reservation_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

In [228]:
#Getting Student Population ratios for TRACT data

#total_student_ratio = university pop / total tract pop 
p5_tract_data_merged['total_student_ratio'] = np.divide(p5_tract_data_merged['P5_008N'], p5_tract_data_merged['total_pop'])
p5_tract_data_merged['total_student_ratio'] = p5_tract_data_merged['total_student_ratio'].replace([np.inf, -np.inf], np.nan).fillna(0)

In [229]:
#new_cols ratios
for col in col_names_tract:
  col_ratio = col + "_ratio"
  p5_tract_data_merged[col_ratio] = np.divide(p5_tract_data_merged[col], p5_tract_data_merged['total_pop'])
  p5_tract_data_merged[col_ratio] = p5_tract_data_merged[col_ratio].replace([np.inf, -np.inf], np.nan).fillna(0)

In [230]:
#Merge County Pop and P5 data 
print(p5_data.info())
print(county_pop_2.info())
p5_data_merged = p5_data.merge(county_pop_2[['GEO_ID', '2020_base']], on='GEO_ID', how='left')
new_cols_county['GEO_ID'] = new_cols_county['GEO_ID'].str[-5:]
new_cols_county['GEO_ID'] = new_cols_county['GEO_ID'].astype(str)

new_cols_county = new_cols_county.drop(new_cols_county.index[0]).reset_index(drop=True)
p5_data_merged['GEO_ID'] = p5_data_merged['GEO_ID'].str.strip()
new_cols_county['GEO_ID'] = new_cols_county['GEO_ID'].str.strip()
for col in col_names_county:
  p5_data_merged = p5_data_merged.merge(new_cols_county[['GEO_ID', col]], on='GEO_ID', how='left')

for column in col_names_county:
    p5_data_merged[column] = pd.to_numeric(p5_data_merged[column], errors='coerce')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 12 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   GEO_ID   3221 non-null   object
 1   NAME     3221 non-null   object
 2   P5_001N  3221 non-null   object
 3   P5_002N  3221 non-null   object
 4   P5_003N  3221 non-null   int64 
 5   P5_004N  3221 non-null   object
 6   P5_005N  3221 non-null   object
 7   P5_006N  3221 non-null   object
 8   P5_007N  3221 non-null   int64 
 9   P5_008N  3221 non-null   int64 
 10  P5_009N  3221 non-null   int64 
 11  P5_010N  3221 non-null   object
dtypes: int64(4), object(8)
memory usage: 302.1+ KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3221 entries, 0 to 3220
Data columns (total 9 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   OBJECTID       3221 non-null   int64  
 1   GEO_ID         3221 non-null   object 
 2   NAME           3221 non-null   

In [231]:
#total_military_ratio = military group quarters population / total county population
p5_data_merged['total_military_ratio'] = p5_data_merged['P5_009N'] / p5_data_merged['2020_base'].replace({0: pd.NA})

#military_ratio = ratio of military population to total non-intstitutionalized population
p5_data_merged['military_ratio'] = p5_data_merged['P5_009N'] / p5_data_merged['P5_007N'].replace({0: pd.NA})

#print(p5_data_merged.head())

In [232]:
#make sure all NaN values are set to 0 BEFORE JOINING TABLE or converting to DICT

p5_data_merged['total_military_ratio'] = p5_data_merged['total_military_ratio'].fillna(0)
p5_data_merged['military_ratio'] = p5_data_merged['military_ratio'].fillna(0)

C:\Users\colin\AppData\Local\Temp\ipykernel_24456\420950401.py:4: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  p5_data_merged['military_ratio'] = p5_data_merged['military_ratio'].fillna(0)


In [233]:
#merge counties shapefile with p5 county data 

merged_counties = p5_data_merged.merge(counties[['GEO_ID', 'STATE_NAME', 'AFFGEOID', 'NAMELSAD', 'geometry']], on='GEO_ID', how='right')
merged_counties['total_incarcerated_ratio'] = np.divide(merged_counties['P5_003N'], merged_counties['2020_base'])
merged_counties['total_student_ratio'] = np.divide(merged_counties['P5_008N'], merged_counties['2020_base'])
for col in col_names_county:
  merged_counties[col + '_ratio']  = np.divide(merged_counties[col], merged_counties['2020_base'])

In [234]:
#merge tract shapefile with p5 TRACT data
merged_tracts = tracts.merge(p5_tract_data_merged[['AFFGEOID','NAME', 'P5_003N', 'P5_009N','P5_011N','total_pop','total_military_ratio','total_incarcerated_ratio', 'relative_military_ratio','total_student_ratio', 'reservation_ratio']], on='AFFGEOID', how='left')
merged_tracts[['total_student_ratio','total_military_ratio','total_incarcerated_ratio', 'reservation_ratio']] = merged_tracts[['total_student_ratio','total_military_ratio','total_incarcerated_ratio','reservation_ratio']].fillna(0)

### Military Tract Data Manipulation ###

In [235]:
#census Tracts + military data manipulation 

#make a copy of merged tracts 
merged_tracts_military = merged_tracts.copy()
merged_tracts_military.reset_index(drop=True, inplace=True)

#set AFFGEOID to 0 for all rows with total military ratio < 2 standard deviations above the mean
mean = np.mean(merged_tracts_military['total_military_ratio'])
std_dev = np.std(merged_tracts_military['total_military_ratio'])
merged_tracts_military.loc[merged_tracts_military['total_military_ratio']<(mean + inclusion_factor * std_dev), 'AFFGEOID'] = 0

In [236]:
#identify which rows we need to dissolve
rows_to_dissolve = merged_tracts_military[merged_tracts_military['AFFGEOID'] == 0]
dissolved_rows = rows_to_dissolve.dissolve(by='AFFGEOID')
dissolved_rows.reset_index(drop=True, inplace=True)

#isolate non-dissolved rows 
non_dissolved_rows = merged_tracts_military[merged_tracts_military['AFFGEOID'] != 0]
non_dissolved_rows.reset_index(drop=True, inplace=True)

#create a column for 'is_dissolved' and assign values
non_dissolved_rows['is_dissolved'] = False

dissolved_rows['is_dissolved'] = True

#concatenate dissolved and non-dissolved rows back together 
military_tract_dissolved = gpd.GeoDataFrame(pd.concat([non_dissolved_rows, dissolved_rows], ignore_index=True))

c:\Users\colin\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [237]:
print(military_tract_dissolved.head())
#print(non_dissolved_rows.head())

    STATE_NAME              AFFGEOID           NAMELSAD         NAMELSADCO  \
0      Florida  1400000US12031013200   Census Tract 132       Duval County   
1  Mississippi  1400000US28047000900     Census Tract 9    Harrison County   
2        Idaho  1400000US16039960500  Census Tract 9605      Elmore County   
3      Alabama  1400000US01101000900     Census Tract 9  Montgomery County   
4      Florida  1400000US12091020800   Census Tract 208    Okaloosa County   

                                            geometry  \
0  POLYGON ((-81.70785 30.20086, -81.70756 30.202...   
1  POLYGON ((-88.94154 30.41357, -88.94012 30.413...   
2  POLYGON ((-115.89577 43.07012, -115.88581 43.0...   
3  POLYGON ((-86.37656 32.39357, -86.37439 32.394...   
4  POLYGON ((-86.79918 30.41663, -86.79893 30.431...   

                                           NAME  P5_003N  P5_009N  P5_011N  \
0       Census Tract 132, Duval County, Florida     15.0    898.0     15.0   
1  Census Tract 9, Harrison County, Mi

### Reservation Tract Data Manipulation ###

In [238]:
#census Tracts + reservation data manipulation 

#make a copy of merged tracts 
merged_tracts_native = merged_tracts.copy()
merged_tracts_native.reset_index(drop=True, inplace=True)

#set AFFGEOID to 0 for all rows with total native ratio < 2 standard deviations above the mean
mean = np.mean(merged_tracts_native['reservation_ratio'])
std_dev = np.std(merged_tracts_native['reservation_ratio'])
merged_tracts_native.loc[merged_tracts_native['reservation_ratio']<(mean + inclusion_factor * std_dev), 'AFFGEOID'] = 0

In [239]:
#identify which rows we need to dissolve
rows_to_dissolve = merged_tracts_native[merged_tracts_native['AFFGEOID'] == 0]
dissolved_rows = rows_to_dissolve.dissolve(by='AFFGEOID')
dissolved_rows.reset_index(drop=True, inplace=True)

#isolate non-dissolved rows 
non_dissolved_rows = merged_tracts_native[merged_tracts_native['AFFGEOID'] != 0]
non_dissolved_rows.reset_index(drop=True, inplace=True)

#create a column for 'is_dissolved' and assign values
non_dissolved_rows['is_dissolved'] = False

dissolved_rows['is_dissolved'] = True

#concatenate dissolved and non-dissolved rows back together 
native_tract_dissolved = gpd.GeoDataFrame(pd.concat([non_dissolved_rows, dissolved_rows], ignore_index=True))

c:\Users\colin\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [240]:
print(native_tract_dissolved.head())

  STATE_NAME              AFFGEOID            NAMELSAD  \
0     Alaska  1400000US02020001731  Census Tract 17.31   
1     Alaska  1400000US02198940100   Census Tract 9401   
2     Alaska  1400000US02185000200      Census Tract 2   
3     Alaska  1400000US02240000100      Census Tract 1   
4    Montana  1400000US30105940600   Census Tract 9406   

                          NAMELSADCO  \
0             Anchorage Municipality   
1  Prince of Wales-Hyder Census Area   
2                North Slope Borough   
3    Southeast Fairbanks Census Area   
4                      Valley County   

                                            geometry  \
0  POLYGON ((-149.74843 61.20576, -149.74842 61.2...   
1  MULTIPOLYGON (((-131.33528 55.1882, -131.33362...   
2  MULTIPOLYGON (((-147.26509 70.21282, -147.2616...   
3  POLYGON ((-145.18996 63.23361, -145.18803 63.2...   
4  POLYGON ((-106.89671 48.37463, -106.89731 48.3...   

                                                NAME  P5_003N  P5_009N  \

### University Tract Data Manipulation ###

In [241]:
#Census tracts + university data manipulation 

#make a copy of the merged tracts
merged_tracts_copy = merged_tracts.copy()
merged_tracts_copy.reset_index(drop=True, inplace=True)

#set AFFGEOID to 0 for all rows with student ratio < 0.5
mean = np.mean(merged_tracts_copy['total_student_ratio'])
std_dev = np.std(merged_tracts_copy['total_student_ratio'])
merged_tracts_copy.loc[merged_tracts_copy['total_student_ratio']<(mean + inclusion_factor * std_dev), 'AFFGEOID'] = 0

#print(merged_tracts_copy.head(40))

In [242]:
rows_to_dissolve = merged_tracts_copy[merged_tracts_copy['AFFGEOID'] == 0]

dissolved_rows = rows_to_dissolve.dissolve(by='AFFGEOID')
dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows = merged_tracts_copy[merged_tracts_copy['AFFGEOID'] != 0]
non_dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows['is_dissolved'] = False

dissolved_rows['is_dissolved'] = True

student_tract_dissolved = gpd.GeoDataFrame(pd.concat([non_dissolved_rows, dissolved_rows], ignore_index=True))

c:\Users\colin\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [243]:
#universities_gdf['university_geometry'] = universities_gdf.geometry

joined_gdf = gpd.sjoin(universities_gdf, non_dissolved_rows, how="inner", predicate='within')

final_gdf = joined_gdf[['UNIVERSITY_NAME', 'ADDRESS', 'CITY', 'STATE', 'COUNTYFIPS','FT_ENROLL','PT_ENROLL','DORM_CAP', 'geometry','AFFGEOID','NAME','total_student_ratio','is_dissolved']]

# #print(final_gdf.head())
# # Resetting indices to ensure they are unique
# final_gdf.reset_index(drop=True, inplace=True)
# final_gdf = gpd.GeoDataFrame(final_gdf, geometry='geometry')

### Incarcerated + Tracts Data Manipulation ###

In [244]:
#Census tracts + incarcerated data manipulation 

#make a copy of the merged tracts
merged_tracts_copy = merged_tracts.copy()
merged_tracts_copy.reset_index(drop=True, inplace=True)

#set AFFGEOID to 0 for all rows with student ratio < 0.5
mean = np.mean(merged_tracts_copy['total_incarcerated_ratio'])
std_dev = np.std(merged_tracts_copy['total_incarcerated_ratio'])
merged_tracts_copy.loc[merged_tracts_copy['total_incarcerated_ratio']<(mean + inclusion_factor * std_dev), 'AFFGEOID'] = 0

In [245]:
#identify which rows we need to dissolve
rows_to_dissolve = merged_tracts_copy[merged_tracts_copy['AFFGEOID'] == 0]

dissolved_rows = rows_to_dissolve.dissolve(by='AFFGEOID')
dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows = merged_tracts_copy[merged_tracts_copy['AFFGEOID'] != 0]
non_dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows['is_dissolved'] = False

dissolved_rows['is_dissolved'] = True

incarcerated_tract_dissolved = gpd.GeoDataFrame(pd.concat([non_dissolved_rows, dissolved_rows], ignore_index=True))

c:\Users\colin\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


In [246]:

#joined_gdf = gpd.sjoin(correctional, non_dissolved_rows, how="inner", predicate='within')

#final_prison_gdf = joined_gdf[['NAME_left', 'POPULATION', 'COUNTYFIPS', 'AFFGEOID', 'total_incarcerated_ratio','is_dissolved','geometry']]


### ACS Tract Data Manipulation ###

In [247]:
ACS_tracts = tracts.merge(ACS_data[['AFFGEOID','DP03_0033PE']], on='AFFGEOID', how='left')

In [248]:
#set AFFGEOID to 0 for all rows with percent population in agriculture, forestry, mining < 0.5
ACS_tracts.loc[ACS_tracts['DP03_0033PE']<50, 'AFFGEOID'] = 0

#identify which rows we need to dissolve
rows_to_dissolve = ACS_tracts[ACS_tracts['AFFGEOID'] == 0]

dissolved_rows = rows_to_dissolve.dissolve(by='AFFGEOID')
dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows = ACS_tracts[ACS_tracts['AFFGEOID'] != 0]
non_dissolved_rows.reset_index(drop=True, inplace=True)

non_dissolved_rows['is_dissolved'] = False

dissolved_rows['is_dissolved'] = True

ACS_tracts_dissolved = gpd.GeoDataFrame(pd.concat([non_dissolved_rows, dissolved_rows], ignore_index=True))

c:\Users\colin\AppData\Local\Programs\Python\Python312\Lib\site-packages\geopandas\geodataframe.py:1819: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  super().__setitem__(key, value)


# 4. Dataset Creation #

Here, the context data is created and exported.

tractInsight holds tract-level data, and countyInsight holds county-level data.

The files containing exported data are county_data.csv and tract_data.csv

In [249]:
tractInsight = tracts[['AFFGEOID', 'STATE_NAME', 'NAMELSADCO', 'NAMELSAD']]


In [250]:
countyInsight = counties[['AFFGEOID', 'STATE_NAME', 'NAMELSAD']]

In [251]:
# TRACTS - Selecting the relevant columns including the joining key 'AFFGEOID'
tract_columns_to_merge = ['AFFGEOID', 'total_pop', 'P5_009N', 'total_military_ratio', 'P5_003N', 'total_incarcerated_ratio', 'P5_008N', 'total_student_ratio', 'P5_011N', 'reservation_ratio']
for col in col_names_tract:
  to_add = [col, col + "_ratio"]
  tract_columns_to_merge.extend(to_add)

merged_tract_data = p5_tract_data_merged[tract_columns_to_merge]

# Merging the dataframes on 'AFFGEOID'
tractInsight = pd.merge(tractInsight, merged_tract_data, on='AFFGEOID', how='left')
tractInsight['AFFGEOID'] = tractInsight['AFFGEOID'].astype(str).str[-11:]

tractInsight.to_csv('temp.csv', index=False)

mean = np.mean(tractInsight['reservation_ratio'])
std_dev = np.std(tractInsight['reservation_ratio'])
tractInsight.loc[tractInsight['reservation_ratio'] < (mean + inclusion_factor * std_dev), ['P5_011N', 'reservation_ratio']] = 0
  
tractInsight = tractInsight.rename(columns={"P5_009N": "total_military", "P5_003N": "total_incarcerated", "P5_008N": "total_student", "P5_011N": "total_reservation", "reservation_ratio": "total_reservation_ratio"})
existing_col_names = ["total_military", "total_incarcerated", "total_student", "total_reservation"]
col_names_tract = existing_col_names + col_names_tract

for col in col_names_tract:
  mean = np.mean(tractInsight[col + "_ratio"])
  std_dev = np.std(tractInsight[col + "_ratio"])
  tractInsight.loc[tractInsight[col + "_ratio"] < (mean + inclusion_factor * std_dev), [col, col + "_ratio"]] = 0
  tractInsight[col + "_ratio"] = tractInsight[col + "_ratio"].fillna(-1)
  tractInsight[col + "_ratio"] = (tractInsight[col + "_ratio"] * 100).apply(lambda x: f"{int(x * 10) / 10:.1f}%")

print(col_names_tract)
print(tractInsight)

['total_military', 'total_incarcerated', 'total_student', 'total_reservation', 'DP03_0024E', 'DP03_0034E']
          AFFGEOID  STATE_NAME          NAMELSADCO              NAMELSAD  \
0      06077003406  California  San Joaquin County    Census Tract 34.06   
1      06077004402  California  San Joaquin County    Census Tract 44.02   
2      06077004600  California  San Joaquin County       Census Tract 46   
3      06077001400  California  San Joaquin County       Census Tract 14   
4      06077003106  California  San Joaquin County    Census Tract 31.06   
...            ...         ...                 ...                   ...   
85182  36027050104    New York     Dutchess County   Census Tract 501.04   
85183  24047950100    Maryland    Worcester County     Census Tract 9501   
85184  17043841310    Illinois       DuPage County  Census Tract 8413.10   
85185  17043846410    Illinois       DuPage County  Census Tract 8464.10   
85186  17043845508    Illinois       DuPage County  Censu

In [252]:
# COUNTIES - Selecting the relevant columns including the joining key 'AFFGEOID'
county_columns_to_merge = ['AFFGEOID', 'total_pop', 'total_military', 'total_military_ratio', 'total_incarcerated', 'total_incarcerated_ratio', 'total_student', 'total_student_ratio']

merged_counties = merged_counties.rename(columns={"P5_009N": "total_military", "P5_003N": "total_incarcerated", "P5_008N": "total_student", "2020_base": "total_pop"})

for col in col_names_county:
  to_add = [col, col + "_ratio"]
  county_columns_to_merge.extend(to_add)

merged_county_data = merged_counties[county_columns_to_merge]

# Merging the dataframes on 'AFFGEOID'
countyInsight = pd.merge(countyInsight, merged_county_data, on='AFFGEOID', how='left')
countyInsight['AFFGEOID'] = countyInsight['AFFGEOID'].astype(str).str[-5:]


In [253]:
#print(merged_county_data)
print(countyInsight)

     AFFGEOID      STATE_NAME          NAMELSAD  total_pop  total_military  \
0       13031         Georgia    Bulloch County    81099.0             0.0   
1       13121         Georgia     Fulton County  1066710.0             0.0   
2       13179         Georgia    Liberty County    65256.0          2766.0   
3       13189         Georgia   McDuffie County    21632.0             0.0   
4       13213         Georgia     Murray County    39973.0             0.0   
...       ...             ...               ...        ...             ...   
3229    21073        Kentucky   Franklin County    51541.0             0.0   
3230    17201        Illinois  Winnebago County   285350.0             0.0   
3231    17123        Illinois   Marshall County    11742.0             0.0   
3232    37185  North Carolina     Warren County    18642.0             0.0   
3233    72051     Puerto Rico  Dorado Municipio    35879.0             0.0   

      total_military_ratio  total_incarcerated  total_incarcera

In [254]:
for col in col_names_county:
  mean = np.mean(countyInsight[col + "_ratio"])
  std_dev = np.std(countyInsight[col + "_ratio"])
  countyInsight.loc[countyInsight[col + "_ratio"] < (mean + inclusion_factor * std_dev), [col, col + "_ratio"]] = 0
  countyInsight[col + "_ratio"] = countyInsight[col + "_ratio"].fillna(-1)
  countyInsight[col + "_ratio"] = (countyInsight[col + "_ratio"] * 100).apply(lambda x: f"{int(x * 10) / 10:.1f}%")

In [255]:
print(countyInsight)

     AFFGEOID      STATE_NAME          NAMELSAD  total_pop  total_military  \
0       13031         Georgia    Bulloch County    81099.0             0.0   
1       13121         Georgia     Fulton County  1066710.0             0.0   
2       13179         Georgia    Liberty County    65256.0          2766.0   
3       13189         Georgia   McDuffie County    21632.0             0.0   
4       13213         Georgia     Murray County    39973.0             0.0   
...       ...             ...               ...        ...             ...   
3229    21073        Kentucky   Franklin County    51541.0             0.0   
3230    17201        Illinois  Winnebago County   285350.0             0.0   
3231    17123        Illinois   Marshall County    11742.0             0.0   
3232    37185  North Carolina     Warren County    18642.0             0.0   
3233    72051     Puerto Rico  Dorado Municipio    35879.0             0.0   

      total_military_ratio  total_incarcerated  total_incarcera

In [256]:
print("Would you like to download county or tract data? Enter 0 for county and 1 for tract.")
# download_type = int(input("Enter here: "))

Would you like to download county or tract data? Enter 0 for county and 1 for tract.


In [257]:
print(countyInsight)

     AFFGEOID      STATE_NAME          NAMELSAD  total_pop  total_military  \
0       13031         Georgia    Bulloch County    81099.0             0.0   
1       13121         Georgia     Fulton County  1066710.0             0.0   
2       13179         Georgia    Liberty County    65256.0          2766.0   
3       13189         Georgia   McDuffie County    21632.0             0.0   
4       13213         Georgia     Murray County    39973.0             0.0   
...       ...             ...               ...        ...             ...   
3229    21073        Kentucky   Franklin County    51541.0             0.0   
3230    17201        Illinois  Winnebago County   285350.0             0.0   
3231    17123        Illinois   Marshall County    11742.0             0.0   
3232    37185  North Carolina     Warren County    18642.0             0.0   
3233    72051     Puerto Rico  Dorado Municipio    35879.0             0.0   

      total_military_ratio  total_incarcerated  total_incarcera

In [258]:
# if download_type == 0: # 1 6 7 9
selected_cols = ['AFFGEOID', 'STATE_NAME', 'NAMELSAD','total_pop']
#for i in range(len(col_names_county)):
  #print(str(i) + ": " + col_names_county[i] + "\n")

# if download_type == 1: # 0 1 2 3
#selected_cols = ['AFFGEOID', 'STATE_NAME', 'NAMELSAD', 'NAMELSADCO','total_pop']
#for i in range(len(col_names_tract)):
  #print(str(i) + ": " + col_names_tract[i] + "\n")

#user_input = input("Enter here: ")
county_list = [1,6,7,9]
tract_list = [0,1,2,3]

#if download_type == 0:
for num in county_list:
  print(num)
  selected_cols.append(col_names_county[num])
  selected_cols.append(col_names_county[num]+"_ratio")
cropped_table = countyInsight[selected_cols]
cropped_table = cropped_table.rename(columns={"AFFGEOID": "FIPS", "STATE_NAME": "State Name", "NAMELSAD": "County Name", "total_pop": "Total Population"})


cols = ['correctional', 'student', 'military', 'native_american']
mask_zero_or_null = cropped_table[cols].isnull() | (cropped_table[cols] == 0)
rows_to_drop = mask_zero_or_null.all(axis=1)
cleaned_df = cropped_table.loc[~rows_to_drop].reset_index(drop=True)

cleaned_df.to_csv('county_data.csv', index=False)

#if download_type == 1:

selected_cols = ['AFFGEOID', 'STATE_NAME', 'NAMELSAD', 'NAMELSADCO','total_pop']

for num in tract_list:
  print(num)
  selected_cols.append(col_names_tract[num])
  selected_cols.append(col_names_tract[num]+"_ratio")
cropped_table = tractInsight[selected_cols]
cropped_table = cropped_table.rename(columns={"AFFGEOID": "FIPS", "STATE_NAME": "State Name", "NAMELSAD": "County Name", "NAMELSADCO": "Tract Name", "total_pop": "Total Population"})

cols = ['total_incarcerated', 'total_student', 'total_military', 'total_reservation']
mask_zero_or_null = cropped_table[cols].isnull() | (cropped_table[cols] == 0)
rows_to_drop = mask_zero_or_null.all(axis=1)
cleaned_df = cropped_table.loc[~rows_to_drop].reset_index(drop=True)
cleaned_df = cleaned_df.rename(columns={"total_incarcerated": "correctional", "total_student": "student", "total_military": "military", "total_reservation": "native_american"})
  
cleaned_df.to_csv('tract_data.csv', index=False)

1
6
7
9
0
1
2
3


In [259]:
# input_file = 'tract_data.csv'
# cropped_table = pd.read_csv(input_file)

# cols = ['total_incarcerated', 'total_student', 'total_military', 'total_reservation']
# mask_zero_or_null = cropped_table[cols].isnull() | (cropped_table[cols] == 0)
# rows_to_drop = mask_zero_or_null.all(axis=1)
# cleaned_df = cropped_table.loc[~rows_to_drop].reset_index(drop=True)

# cleaned_df.to_csv('tract_data.csv', index=False)

# 5. Visualization #

Here, you can visualize exported data. If you would just like to view exported data and not view visualized data, then this section is irrelevant. The section will prompt you to choose which data to visualize. You can only select one dataset. The map will be displayed on the bottom.

## DEFUNCT - Everything below this line has deprecated functionality. It is here for future development, but will not work.

In [260]:
print("Would you like to visualize county or tract data? Enter 0 for county and 1 for tract.")
download_type = int(input("Enter here: "))

Would you like to visualize county or tract data? Enter 0 for county and 1 for tract.


ValueError: invalid literal for int() with base 10: ''

In [546]:
print("Please select a dataset to visualize. To select a dataset, enter the index assigned to the left of the dataset name. The datasets are as follows:\n")

if download_type == 0:
  selected_cols = ['AFFGEOID', 'STATE_NAME','NAMELSAD','total_pop']
  for i in range(len(col_names_county)):
    print(str(i) + ": " + col_names_county[i] + "\n")

if download_type == 1:
  selected_cols = ['AFFGEOID', 'STATE_NAME', 'NAMELSADCO', 'NAMELSAD','total_pop']
  for i in range(len(col_names_tract)):
    print(str(i) + ": " + col_names_tract[i] + "\n")

user_input = input("Enter here: ")

Please select a dataset to visualize. To select a dataset, enter the index assigned to the left of the dataset name. The datasets are as follows:

0: institutionalized

1: correctional

2: juvenile

3: nursing

4: otherins

5: nonins

6: student

7: military

8: othernonins

9: native_american



## a. Dictionaries ##

In [547]:
if download_type == 0:
  selected_dict = countyInsight.set_index("AFFGEOID")[col_names_county[int(user_input)]+"_ratio"]
else:
  selected_dict = tractInsight.set_index("AFFGEOID")[col_names_tract[int(user_input)]+"_ratio"]
selected_dict = {key: float(value) for key, value in selected_dict.items()}

ValueError: could not convert string to float: '0.0%'

In [ ]:
if download_type == 0:
  counties['AFFGEOID'] = counties['AFFGEOID'].astype(str).str[-5:]
  selected_data = counties.merge(countyInsight[['AFFGEOID','total_pop', col_names_county[int(user_input)], col_names_county[int(user_input)]+"_ratio"]], on='AFFGEOID', how='left')
  selected_data['AFFGEOID'] = selected_data['AFFGEOID'].astype(str).str[-5:]
else:
  tracts['AFFGEOID'] = tracts['AFFGEOID'].astype(str).str[-5:]
  selected_data = tracts.merge(tractInsight[['AFFGEOID','total_pop', col_names_tract[int(user_input)], col_names_tract[int(user_input)]+"_ratio"]], on='AFFGEOID', how='left')
  selected_data['AFFGEOID'] = selected_data['AFFGEOID'].astype(str).str[-5:]


## c. GeoJson conversion ##

In [ ]:
#convert to geojson as the last step before mapping 
selected_geojson = selected_data.to_json()

## d. Popups + Tooltips

In [ ]:
#SELECTED DATA POPUP
if download_type == 0:
    selected_data_popup = folium.GeoJsonPopup(
        fields=["NAMELSAD","total_pop", col_names_county[int(user_input)], col_names_county[int(user_input)]+"_ratio"],
        aliases=["County Name", "Population", col_names_county[int(user_input)], col_names_county[int(user_input)] + " Ratio"],
        localize=True,
        labels=True,
        style="background-color: yellow;",
    )

    selected_data_tooltip = folium.GeoJsonTooltip(
        fields=["NAMELSAD","total_pop", col_names_county[int(user_input)], col_names_county[int(user_input)]+"_ratio"],
        aliases=["County Name",
                 "Population", col_names_county[int(user_input)],
                 col_names_county[int(user_input)] + " Ratio"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )
else:
    selected_data_popup = folium.GeoJsonPopup(
        fields=["NAMELSADCO", "NAMELSAD","total_pop", col_names_tract[int(user_input)], col_names_tract[int(user_input)]+"_ratio"],
        aliases=["County Name", "Tract Name", "Population", col_names_tract[int(user_input)], col_names_tract[int(user_input)] + " Ratio"],
        localize=True,
        labels=True,
        style="background-color: yellow;",
    )

    selected_data_tooltip = folium.GeoJsonTooltip(
        fields=["NAMELSADCO", "NAMELSAD","total_pop", col_names_tract[int(user_input)], col_names_tract[int(user_input)]+"_ratio"],
        aliases=["County Name",
                 "Tract Name",
                 "Population", col_names_tract[int(user_input)],
                 col_names_tract[int(user_input)] + " Ratio"],
        localize=True,
        sticky=False,
        labels=True,
        style="""
            background-color: #F0EFEF;
            border: 2px solid black;
            border-radius: 3px;
            box-shadow: 3px;
        """,
        max_width=800,
    )

## Extra Changes

## e. Mapping ##

In [ ]:
#Selected Data Map

central_lat = 39.8283
central_lon = -98.5795

selected_data_map = folium.Map(location=[central_lat, central_lon], zoom_start=4)

# Add the GeoJSON layer with style

if download_type == 0:
    folium.GeoJson(
        selected_geojson,
        style_function=lambda feature: {
            "fillColor": "red" if feature['properties'].get(col_names_county[int(user_input)], None) not in [0, None] else "white",
            "color": "black" if feature['properties'].get(col_names_county[int(user_input)], None) not in [0, None] else None,
            "weight": 1 if feature['properties'].get(col_names_county[int(user_input)], None) not in [0, None] else 0,
            "fillOpacity": 0.9 if feature['properties'].get(col_names_county[int(user_input)], None) not in [0, None] else 0,
        },
        tooltip=selected_data_tooltip,  # Tooltip to display additional information
        popup=selected_data_popup,     # Popup for detailed information
    ).add_to(selected_data_map)
else:
    folium.GeoJson(
        selected_geojson,
        style_function=lambda feature: {
            "fillColor": "red" if feature['properties'].get(col_names_tract[int(user_input)], None) not in [0, None] else "white",
            "color": "black" if feature['properties'].get(col_names_tract[int(user_input)], None) not in [0, None] else None,
            "weight": 1 if feature['properties'].get(col_names_tract[int(user_input)], None) not in [0, None] else 0,
            "fillOpacity": 0.9 if feature['properties'].get(col_names_tract[int(user_input)], None) not in [0, None] else 0,
        },
        tooltip=selected_data_tooltip,  # Tooltip to display additional information
        popup=selected_data_popup,     # Popup for detailed information
    ).add_to(selected_data_map)
selected_data_map